In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import remote_control

remote_control.__file__

In [ ]:
from remote_control.control import configure_fly_at_fixed_z
configure_fly_at_fixed_z(2000, 500)

In [ ]:
from remote_control import acquisition
slides = acquisition.SLIDES
#print(slides)
custom_plate =  { 
    "name": "labtek8",
    "spot_spacing": (10.5, 12.5, 1), #hor,ver (mm) centre to centre distance
    "spot_size": (4.5 ,7, 1), #h, v (mm)
    "grid_size": (2,4), # h, v,
    "shape": "rectangle",
}

ga = acquisition.WellPlateGridAquisition(plate_type=custom_plate, 
                                         config_fn = 'remote_config.json', 
                                         datadir="./data", # will save spatial position file here
                                         log_fname="./test.log")

instrument_positions = [
         (422, 4252, -4.4),
         (-10252, 4187, -94.5),
         (29, 42684, 122.5),
         (-10536, 42666, 43)
]
#wells for calibration
wells = [
    (0,0,0),
    (1,0,0),
    (0,3,0),
    (1,3,0),
] 

image_bounds = [
                  [3700, -450],
                  [-13900, 48000],
                ] # [.max_x, min_y] [min_x, .max_y] 

ga.calibrate(instrument_positions, wells, 'centre') #calibration point location!
ga.set_image_bounds(image_bounds)

#plt.figure()
#plt.scatter([c[0] for c in instrument_positions], [c[1] for c in instrument_positions])
#plt.show()

#plt.figure()
#plt.scatter([c[0] for c in instrument_positions], [c[1] for c in wells])
#plt.show()

spotsToUse = [
(0, 0),
(1, 0),
(0, 1),
(1, 1),
(0, 2), 
(1, 2),
(0, 3),
(1, 3)    
]

# Generate coordinates & preview
ga.generate_targets(spotsToUse, offset_x=0, offset_y=0, pixelsize_x=50, pixelsize_y=50, area_function_name='lower_right')
ga.plot_targets()

# Write coordinates to file locally

In [ ]:
ga.write_imzc_coords_file('./data/positions_imzc.txt')
ga.write_json_coords_file('./data/positions.json')

# Perform acquisition & write to file in output directory

In [ ]:
filename = "./data/01052019_Mouse_DHB_pos_mz200-800_px50x50_LR" # Prefix for output files

ga.acquire(filename, dummy=True)